In [1]:
import json
import os

In [2]:
annotations_save_root = os.path.join(os.getcwd(), 'dataset', 'training_data', 'annotations')

In [3]:
# Открываем файл полученный из label-studio (export format: json-min)
with open('/home/berkiu/work/project-2-at-2022-10-06-11-15-c47ffc3c.json', 'r') as f:
    data = json.load(f)

In [4]:
class PrepareAnnotations():
    def __init__(self, labeled_json_root):
        self.ner_to_class = {'O': 'other',
                             'B-QUESTION': 'question',
                             'I-QUESTION': 'question',
                             'B-ANSWER': 'answer',
                             'I-ANSWER': 'answer',
                             'B-HEADER': 'header',
                             'I-HEADER': 'header'}
        with open(labeled_json_root, 'r') as f:
            self.data = json.load(f)
    
    @staticmethod
    def unnormalize_bbox(bbox, width, height):
        """Returns unnormalized bbox [x_0, y_0, x_1, y_1] format."""
        x_0 = bbox[0] / 100 * width
        y_0 = bbox[1] / 100 * height
        x_1 = x_0 + bbox[2] / 100 * width
        y_1 = y_0 + bbox[3] / 100 * height
        return [x_0, y_0, x_1, y_1]
    
    def get_one_class(self, words, text, counter):
        one_class = {}
        min_x_0 = sorted(words, key=lambda x: x['box'][0])[0]['box'][0]
        min_y_0 = sorted(words, key=lambda x: x['box'][1])[0]['box'][1]
        max_x_1 = sorted(words, key=lambda x: x['box'][2])[-1]['box'][2]
        max_y_1 = sorted(words, key=lambda x: x['box'][3])[-1]['box'][3]
        one_class['box'] = [min_x_0, min_y_0, max_x_1, max_y_1]
        one_class['text'] = text
        one_class['label'] = self.ner_to_class[current_label]
        one_class['words'] = words
        one_class['id'] = counter
        return one_class

    def get_annotations(self):
        for file in self.data:
            file_name = file['ocr'][file['ocr'].find('-') + 1:file['ocr'].rfind('.')]
            annotation = {'form': []}
            words = []
            text = ''
            counter = 0
            original_width = file['label'][0]['original_width']
            original_height = file['label'][0]['original_height']
            for idx, item in enumerate(file['label']):
                current_label = item['labels'][0]
                current_word = file['transcription'][idx]
                current_bbox = [item['x'], item['y'], item['width'], item['height']]
                unnormalized_bbox = self.unnormalize_bbox(current_bbox, original_width, original_height)
                text += current_word
                words.append({'text': current_word, 'box': unnormalized_bbox})
                if idx != (len(file['label']) - 1):
                    next_label = file['label'][idx + 1]['labels'][0]
                else:
                    one_class = get_one_class(words, text, counter)
                    annotation['form'].append(one_class)
                    break
                if (next_label == current_label and ('B-' not in current_label and 'B-' not in next_label)) or ('B-' in current_label and 'I-' in next_label and (current_label[1:] == next_label[1:])):
                    text += ' '
                    continue
                else:
                    one_class = get_one_class(words, text, counter)
                    annotation['form'].append(one_class)
                    counter += 1
                    words = []
                    text = ''
            with open(os.path.join(annotations_save_root, f'{file_name}.json'), 'w') as f:
                json.dump(annotation, f)

In [5]:
preparator = PrepareAnnotations('/home/berkiu/work/project-2-at-2022-10-06-11-15-c47ffc3c.json')

In [23]:
preparator.get_annotations()